In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
CPSData = pd.read_csv('./data/CPSData.csv')

In [3]:
CPSData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131302 entries, 0 to 131301
Data columns (total 14 columns):
PeopleInHousehold     131302 non-null int64
Region                131302 non-null object
State                 131302 non-null object
MetroAreaCode         97064 non-null float64
Age                   131302 non-null int64
Married               105964 non-null object
Sex                   131302 non-null object
Education             105964 non-null object
Race                  131302 non-null object
Hispanic              131302 non-null int64
CountryOfBirthCode    131302 non-null int64
Citizenship           131302 non-null object
EmploymentStatus      105513 non-null object
Industry              66242 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 10.5+ MB


In [4]:
# Most common industry of employment among interviewees
CPSData.Industry.value_counts().order(ascending=False)

Educational and health services                15017
Trade                                           8933
Professional and business services              7519
Manufacturing                                   6791
Leisure and hospitality                         6364
Construction                                    4387
Financial                                       4347
Transportation and utilities                    3260
Other services                                  3224
Public administration                           3186
Information                                     1328
Agriculture, forestry, fishing, and hunting     1307
Mining                                           550
Armed forces                                      29
dtype: int64

In [7]:
# Which state has the fewest interviewees ?
CPSData.State.value_counts().order().head(1)

New Mexico    1102
dtype: int64

In [8]:
# Which state has the largest number of interviewees ?
CPSData.State.value_counts().order(ascending=False).head(1)

California    11570
dtype: int64

In [14]:
# What proportion of interviewees are citizens of the United States ?
citizen = CPSData[CPSData.Citizenship != 'Non-Citizen']
print (len(citizen) * 1.) / (CPSData.shape[0])

0.942194330627


In [33]:
# For which race there are atleast 250 interviewees in the CPS dataset of Hispanic ethnicity ?
c = CPSData.groupby(['Race', 'Hispanic']).size().unstack(level=1)
c[c[1] > 250]

Hispanic,0,1
Race,,
American Indian,1129,304
Black,13292,621
Multiracial,2449,448
White,89190,16731


In [22]:
c.head()

Race             Hispanic
American Indian  0            1129
                 1             304
Asian            0            6407
                 1             113
Black            0           13292
dtype: int64

### Evaluating Missing Values

In [34]:
# Which variables have at least one interviewee with a missing (NA) value ?
CPSData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131302 entries, 0 to 131301
Data columns (total 14 columns):
PeopleInHousehold     131302 non-null int64
Region                131302 non-null object
State                 131302 non-null object
MetroAreaCode         97064 non-null float64
Age                   131302 non-null int64
Married               105964 non-null object
Sex                   131302 non-null object
Education             105964 non-null object
Race                  131302 non-null object
Hispanic              131302 non-null int64
CountryOfBirthCode    131302 non-null int64
Citizenship           131302 non-null object
EmploymentStatus      105513 non-null object
Industry              66242 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 10.5+ MB


In [36]:
# Identify the pattern in the missing values in the dataset?
CPSData.groupby(['Region', CPSData.Married.isnull()]).size()

Region     Married
Midwest    False      24609
           True        6075
Northeast  False      21432
           True        4507
South      False      33535
           True        7967
West       False      26388
           True        6789
dtype: int64

In [37]:
CPSData.groupby(['Sex', CPSData.Married.isnull()]).size()

Sex     Married
Female  False      55264
        True       12217
Male    False      50700
        True       13121
dtype: int64

In [38]:
CPSData.groupby(['Age', CPSData.Married.isnull()]).size()

Age  Married
0    True       1283
1    True       1559
2    True       1574
3    True       1693
4    True       1695
5    True       1795
6    True       1721
7    True       1681
8    True       1729
9    True       1748
10   True       1750
11   True       1721
12   True       1797
13   True       1802
14   True       1790
...
67   False      1227
68   False      1130
69   False      1062
70   False      1195
71   False      1031
72   False       941
73   False       896
74   False       842
75   False       763
76   False       729
77   False       698
78   False       659
79   False       661
80   False      2664
85   False      2446
Length: 82, dtype: int64

In [39]:
CPSData.groupby(['Citizenship', CPSData.Married.isnull()]).size()

Citizenship           Married
Citizen, Native       False      91956
                      True       24683
Citizen, Naturalized  False       6910
                      True         163
Non-Citizen           False       7098
                      True         492
dtype: int64

In [70]:
# How many states had all the interviewees living in a non-metropolitan area?
# Treat District of Columbia as a state, even though it is not technically a state ?
c = CPSData.groupby(['State', CPSData.MetroAreaCode.isnull()]).size().unstack()

c.head()

MetroAreaCode,False,True
State,,
Alabama,1020,356
Alaska,NaN,1590
Arizona,1327,201
Arkansas,724,697
California,11333,237


In [66]:
# How many states have all interviewees living in a non-metropolitan area ?
list(pd.isnull(c[c.columns[0]])).count(True)

2

In [68]:
# How many states have all the interviewees living in a metropolitan area ?
list(pd.isnull(c[c.columns[1]])).count(True)

3

In [76]:
# Which region of US has the largest proportion of interviewees living in a non-metropolitan area
c = CPSData.groupby(['Region', CPSData.MetroAreaCode.isnull()]).size().unstack()
c.head()

MetroAreaCode,False,True
Region,,
Midwest,20010,10674
Northeast,20330,5609
South,31631,9871
West,25093,8084


In [78]:
c['prop_non_metro'] = c[c.columns[1]] / (c[c.columns[1]] + c[c.columns[0]])

In [79]:
c.prop_non_metro.order(ascending=False)

Region
Midwest      0.347869
West         0.243663
South        0.237844
Northeast    0.216238
Name: prop_non_metro, dtype: float64

In [94]:
# Which state has a proportion of interviewees living in a non-metropolitan area closest to 30%?
c = CPSData.groupby('State')
c['MetroAreaCode'].apply(lambda x: np.mean(x.isnull()))

State
Alabama                 0.258721
Alaska                  1.000000
Arizona                 0.131545
Arkansas                0.490500
California              0.020484
Colorado                0.129915
Connecticut             0.085684
Delaware                0.233966
District of Columbia    0.000000
Florida                 0.039231
Georgia                 0.198432
Hawaii                  0.249166
Idaho                   0.498682
Illinois                0.112219
Indiana                 0.291417
Iowa                    0.486946
Kansas                  0.362274
Kentucky                0.506790
Louisiana               0.161379
Maine                   0.598321
Maryland                0.069375
Massachusetts           0.064922
Michigan                0.178257
Minnesota               0.315068
Mississippi             0.694309
Missouri                0.328671
Montana                 0.836079
Nebraska                0.581324
Nevada                  0.133082
New Hampshire           0.568745
New 

In [95]:
c = CPSData.groupby('State')
c['MetroAreaCode'].apply(lambda x: np.mean(x.isnull())).order(ascending=False)

State
Wyoming                 1.000000
Alaska                  1.000000
Montana                 0.836079
West Virginia           0.755855
North Dakota            0.737386
South Dakota            0.702500
Mississippi             0.694309
Vermont                 0.652381
Maine                   0.598321
Nebraska                0.581324
New Hampshire           0.568745
Kentucky                0.506790
Idaho                   0.498682
Arkansas                0.490500
Iowa                    0.486946
North Carolina          0.373043
Kansas                  0.362274
Tennessee               0.355942
Missouri                0.328671
Oklahoma                0.327643
Minnesota               0.315068
South Carolina          0.313028
Wisconsin               0.299330
Indiana                 0.291417
Alabama                 0.258721
Ohio                    0.251223
Hawaii                  0.249166
New Mexico              0.245009
Delaware                0.233966
Oregon                  0.218219
Utah

In [96]:
MetroAreaMap = pd.read_csv('./data/MetroAreaCodes.csv')
CountryMap = pd.read_csv('./data/CountryCodes.csv')

In [97]:
MetroAreaMap.shape

(271, 2)

In [98]:
CountryMap.shape

(149, 2)

In [101]:
cps_with_metro_names = pd.merge(CPSData, MetroAreaMap, left_on='MetroAreaCode', right_on='Code', how='left')

In [105]:
cps_with_metro_names.columns

Index([u'PeopleInHousehold', u'Region', u'State', u'MetroAreaCode', u'Age', u'Married', u'Sex', u'Education', u'Race', u'Hispanic', u'CountryOfBirthCode', u'Citizenship', u'EmploymentStatus', u'Industry', u'Code', u'MetroArea'], dtype='object')

In [110]:
list(cps_with_metro_names.MetroArea.isnull()).count(True)

34238

In [111]:
cps_with_metro_names.groupby('MetroArea').size().order(ascending=False)

MetroArea
New York-Northern New Jersey-Long Island, NY-NJ-PA    5409
Washington-Arlington-Alexandria, DC-VA-MD-WV          4177
Los Angeles-Long Beach-Santa Ana, CA                  4102
Philadelphia-Camden-Wilmington, PA-NJ-DE              2855
Chicago-Naperville-Joliet, IN-IN-WI                   2772
Providence-Fall River-Warwick, MA-RI                  2284
Boston-Cambridge-Quincy, MA-NH                        2229
Minneapolis-St Paul-Bloomington, MN-WI                1942
Dallas-Fort Worth-Arlington, TX                       1863
Houston-Baytown-Sugar Land, TX                        1649
Honolulu, HI                                          1576
Miami-Fort Lauderdale-Miami Beach, FL                 1554
Atlanta-Sandy Springs-Marietta, GA                    1552
Denver-Aurora, CO                                     1504
Baltimore-Towson, MD                                  1483
...
Vineland-Millville-Bridgeton, NJ    54
Prescott, AZ                        54
Santa Fe, NM           

In [116]:
c = cps_with_metro_names.groupby('MetroArea')
c['Hispanic'].apply(lambda x: (list(x).count(1) * 1. / (list(x).count(1) + list(x).count(0)))).order(ascending=False)

MetroArea
Laredo, TX                               0.966292
McAllen-Edinburg-Pharr, TX               0.948718
Brownsville-Harlingen, TX                0.797468
El Paso, TX                              0.790984
El Centro, CA                            0.686869
San Antonio, TX                          0.644152
Madera, CA                               0.614035
Corpus Christi, TX                       0.606061
Merced, CA                               0.566038
Salinas, CA                              0.557692
Las Cruses, NM                           0.542056
Tucson, AZ                               0.506623
Riverside-San Bernardino, CA             0.502326
Bakersfield, CA                          0.489796
Miami-Fort Lauderdale-Miami Beach, FL    0.467825
...
Anderson, SC                     0
Ann Arbor, MI                    0
Macon, GA                        0
Savannah, GA                     0
Johnstown, PA                    0
Hagerstown-Martinsburg, MD-WV    0
Harrisonburg, VA          

In [125]:
c = cps_with_metro_names.groupby('MetroArea')
prop_asians = lambda z: (list(z['Race']).count('Asian') * 1.) / z['Race'].shape[0]
c.filter(lambda x: prop_asians(x) > .2).MetroArea.unique()

array(['San Francisco-Oakland-Fremont, CA',
       'San Jose-Sunnyvale-Santa Clara, CA', 'Vallejo-Fairfield, CA',
       'Honolulu, HI'], dtype=object)

In [121]:
cps_with_metro_names.Race.head()

0    White
1    Black
2    Black
3    Black
4    White
Name: Race, dtype: object

In [129]:
# Which metropolitan area has the smallest proportion of interviewees who have received no high school diploma?
c['Education'].apply(lambda x: list(x).count('No high school diploma') * 1. / x.shape[0]).order()

MetroArea
Iowa City, IA                        0.022901
Bowling Green, KY                    0.034483
Kalamazoo-Portage, MI                0.039370
Champaign-Urbana, IL                 0.040984
Bremerton-Silverdale, WA             0.045977
Jacksonville, NC                     0.047619
Bloomington-Normal IL                0.050000
Lawrence, KS                         0.051020
Salisbury, MD                        0.054054
Eau Claire, WI                       0.054545
Gainesville, FL                      0.057143
Fort Collins-Loveland, CO            0.058252
Palm Bay-Melbourne-Titusville, FL    0.059524
Madison, WI                          0.059859
Fargo, ND-MN                         0.060185
...
Montgomery, AL                0.203883
Tucson, AZ                    0.205298
Lawton, OK                    0.216495
Lancaster, PA                 0.217949
El Paso, TX                   0.225410
Corpus Christi, TX            0.227273
Laredo, TX                    0.235955
Madera, CA             

In [130]:
CountryMap.columns

Index([u'Code', u'Country'], dtype='object')

In [131]:
cps_with_metro_names.columns

Index([u'PeopleInHousehold', u'Region', u'State', u'MetroAreaCode', u'Age', u'Married', u'Sex', u'Education', u'Race', u'Hispanic', u'CountryOfBirthCode', u'Citizenship', u'EmploymentStatus', u'Industry', u'Code', u'MetroArea'], dtype='object')

In [132]:
cps_with_metro_and_country_names = pd.merge(cps_with_metro_names, 
                                            CountryMap, 
                                            left_on='CountryOfBirthCode',
                                            right_on='Code',
                                            how='left'
                                           )

In [133]:
cps_with_metro_and_country_names.columns

Index([u'PeopleInHousehold', u'Region', u'State', u'MetroAreaCode', u'Age', u'Married', u'Sex', u'Education', u'Race', u'Hispanic', u'CountryOfBirthCode', u'Citizenship', u'EmploymentStatus', u'Industry', u'Code_x', u'MetroArea', u'Code_y', u'Country'], dtype='object')

In [136]:
cps_with_metro_and_country_names.Country.isnull().sum()

176

In [138]:
# Among all interviewees born outside of North America, which country was the most common place of birth?
cps_with_metro_and_country_names.groupby('Country').size().order(ascending=False)

Country
United States         115063
Mexico                  3921
Philippines              839
India                    770
China                    581
Puerto Rico              518
El Salvador              477
Vietnam                  458
Germany                  438
Cuba                     426
Canada                   410
Korea                    334
Dominican Republic       330
Guatemala                309
Jamaica                  217
...
Barbados            6
Tonga               6
Slovakia            6
Denmark             6
Singapore           6
Senegal             6
Samoa               6
Latvia              6
Zimbabwe            6
Georgia             5
Azerbaijan          3
St. Kitts--Nevis    3
Czechoslovakia      3
Tanzania            2
Northern Ireland    2
Length: 144, dtype: int64

In [147]:
# What proportion of the interviewees from the "New York-Northern New Jersey-Long Island, 
# NY-NJ-PA" metropolitan area have a country of birth that is not the United States? 
# For this computation, don't include people from this metropolitan area who have a missing country of birth.

c = cps_with_metro_and_country_names[cps_with_metro_and_country_names.MetroArea == 
                                     'New York-Northern New Jersey-Long Island, NY-NJ-PA']

In [152]:
c = c[c.Country.notnull()]

In [155]:
c[c.Country != 'United States'].shape[0] * 1. / c.shape[0] 

0.30866025166543304

In [156]:
# Which metropolitan area has the largest number (note -- not proportion) of interviewees with a country of birth in India?
c = cps_with_metro_and_country_names.groupby('MetroArea')
c['Country'].apply(lambda x: x[(x.notnull()) & (x == 'India')].shape[0]).order(ascending=False).head(1)

MetroArea
New York-Northern New Jersey-Long Island, NY-NJ-PA    96
Name: Country, dtype: int64

In [157]:
c = cps_with_metro_and_country_names.groupby('MetroArea')
c['Country'].apply(lambda x: x[(x.notnull()) & (x == 'Brazil')].shape[0]).order(ascending=False).head(1)

MetroArea
Boston-Cambridge-Quincy, MA-NH    18
Name: Country, dtype: int64

In [158]:
c = cps_with_metro_and_country_names.groupby('MetroArea')
c['Country'].apply(lambda x: x[(x.notnull()) & (x == 'Somalia')].shape[0]).order(ascending=False).head(1)

MetroArea
Minneapolis-St Paul-Bloomington, MN-WI    17
Name: Country, dtype: int64